## Example of Warm Starting Transformers

source: https://huggingface.co/blog/warm-starting-encoder-decoder



In [43]:

## !pip install git-python==1.0.3
## !pip install rouge_score
## !pip install sacrebleu



In [52]:

import transformers
import datasets
import pandas as pd

from IPython.display import display, HTML
from datasets import ClassLabel
from transformers import BertTokenizerFast
from transformers import BertTokenizer




## This are the imports for the Transfomer Encoder Decoder with Multihead Attention to be Warm-Started with an Encoder and a GPT for the Decoder


In [45]:


from transformers import EncoderDecoderModel
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments



In [18]:


sample_size         = 10000
encoder_max_length  = 512
decoder_max_length  = 128


# batch_size = 16
batch_size=4



In [2]:

train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /Users/user/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


In [3]:

train_data.info.description



'CNN/DailyMail non-anonymized summarization dataset.\n\nThere are two features:\n  - article: text of news article, used as the document to be summarized\n  - highlights: joined text of highlights with <s> and </s> around each\n    highlight, which is the target summary\n'

In [7]:




df = pd.DataFrame(   train_data[:3]      )

del df["id"]

for column, typ in train_data.features.items():
      if isinstance(typ, ClassLabel):
          df[column] = df[column].transform(lambda i: typ.names[i])
        
display(HTML(df.to_html()))





,article,highlights
0,"LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in ""Harry Potter and the Order of the Phoenix"" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. ""I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar,"" he told an Australian interviewer earlier this month. ""I don't think I'll be particularly extravagant. ""The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs."" At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film ""Hostel: Part II,"" currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. ""I'll definitely have some sort of party,"" he said in an interview. ""Hopefully none of you will be reading about it."" Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. ""People are always looking to say 'kid star goes off the rails,'"" he told reporters last month. ""But I try very hard not to go that way because it would be too easy for them."" His latest outing as the boy wizard in ""Harry Potter and the Order of the Phoenix"" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called ""My Boy Jack,"" about author Rudyard Kipling and his son, due for release later this year. He will also appear in ""December Boys,"" an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's ""Equus."" Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: ""I just think I'm going to be more sort of fair game,"" he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed.",Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund .
1,"Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the ""forgotten floor,"" where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the ""forgotten floor."" Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually ""avoidable felonies."" He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to follow directions, according to Leifman. So, they end up on the ninth floor severely mentally disturbed, but not getting any real help because they're in jail. We toured the jail with Leifman. He is well

In [9]:

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")




## Map article and summary len to dict as well as if sample is longer than 512 tokens


In [10]:


def map_to_length(x):
  x["article_len"]        = len(tokenizer(x["article"]).input_ids)
  x["article_longer_512"] = int(x["article_len"] > 512)
    
  x["summary_len"]        = len(tokenizer(x["highlights"]).input_ids)
  x["summary_longer_64"]  = int(x["summary_len"] > 64)
  x["summary_longer_128"] = int(x["summary_len"] > 128)
  return x




In [12]:


data_stats = train_data.select(range(sample_size)).map(map_to_length, num_proc=4)



Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (587 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (779 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (963 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1341 > 512). Running this sequence through the model will result in indexing errors


In [13]:

def compute_and_print_stats(x):
    
  if len(x["article_len"]) == sample_size:
    print(
        "Article Mean: {}, %-Articles > 512:{}, Summary Mean:{}, %-Summary > 64:{}, %-Summary > 128:{}".format(
            sum(x["article_len"]) / sample_size,
            sum(x["article_longer_512"]) / sample_size, 
            sum(x["summary_len"]) / sample_size,
            sum(x["summary_longer_64"]) / sample_size,
            sum(x["summary_longer_128"]) / sample_size,
        )
    )







In [14]:

output = data_stats.map(
  compute_and_print_stats, 
  batched=True,
  batch_size=-1,
)


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Article Mean: 805.7411, %-Articles > 512:0.7226, Summary Mean:57.2474, %-Summary > 64:0.2652, %-Summary > 128:0.0




We can see that on average an article contains 806 tokens with ca. 3/4 of the articles being longer than the model's max_length 512. 

The summary is on average 57 tokens long. Over 26% of our 10000-sample summaries are longer than 64 tokens, but none are longer than 128 tokens.



In [16]:



def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs  = tokenizer(batch["article"],    padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["highlights"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"]      = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["labels"]         = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
    
  batch["labels"] = [
     [-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
  ]

  return batch




In [17]:


train_data = train_data.select(    range(32)   )



In [19]:


train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)



Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [20]:

train_data


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 32
})

In [21]:

train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)



In [23]:

val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")




Found cached dataset cnn_dailymail (/Users/user/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


In [24]:

val_data = val_data.select(range(8))



In [25]:

val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["article", "highlights", "id"]
)


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [27]:

val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"],
)



In [32]:

bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [33]:

bert2bert



EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [34]:

bert2bert.save_pretrained("bert2bert")



In [35]:

bert2bert = EncoderDecoderModel.from_pretrained("bert2bert")



In [36]:

bert2bert.config



EncoderDecoderConfig {
  "_commit_hash": null,
  "_name_or_path": "bert2bert",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "bert-base-uncased",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
     

In [60]:


bert2bert.config.decoder_start_token_id = tokenizer.cls_token_id
bert2bert.config.eos_token_id           = tokenizer.sep_token_id
bert2bert.config.pad_token_id           = tokenizer.pad_token_id
bert2bert.config.vocab_size             = bert2bert.config.encoder.vocab_size



In [61]:

bert2bert.config.max_length           = 142
bert2bert.config.min_length           = 56
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping       = True
bert2bert.config.length_penalty       = 2.0
bert2bert.config.num_beams            = 4



## Fine-Tuning Warm-Started Encoder-Decoder Models


In [62]:

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size =batch_size,
    ## fp16=True,   ## GPU only 
    output_dir="./",
    logging_steps=2,
    save_steps=10,
    eval_steps=4,
    # logging_steps=1000,
    # save_steps=500,
    # eval_steps=7500,
    # warmup_steps=2000,
    # save_total_limit=3,
)




In [63]:

rouge = datasets.load_metric("rouge")


In [64]:

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids   = pred.predictions

    pred_str                       = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str                      = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }



In [65]:


trainer = Seq2SeqTrainer(
    model           = bert2bert,
    tokenizer       = tokenizer,
    args            = training_args,
    compute_metrics = compute_metrics,
    train_dataset   = train_data,
    eval_dataset    = val_data,
)


In [66]:

trainer.train()


/Users/user/opt/anaconda3/envs/py37_HF2_Transformers/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/Users/user/opt/anaconda3/envs/py37_HF2_Transformers/lib/python3.7/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
4,1.771200,2.750534,0.034500,0.065100,0.044100


TrainOutput(global_step=6, training_loss=1.8677761952082317, metrics={'train_runtime': 1226.5462, 'train_samples_per_second': 0.078, 'train_steps_per_second': 0.005, 'total_flos': 58891593646080.0, 'train_loss': 1.8677761952082317, 'epoch': 3.0})

In [71]:

!ls


bert2bert                    warmStarting_Bert2Bert.ipynb


In [72]:

## dummy_bert2bert = EncoderDecoderModel.from_pretrained("./checkpoint-20")



## Evaluation


In [73]:



bert2bert = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail") ## .to("cuda")
tokenizer = BertTokenizer.from_pretrained(      "patrickvonplaten/bert2bert_cnn_daily_mail")




In [74]:

test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test[:2%]")


In [75]:

def generate_summary(batch):
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, 
                             return_tensors="pt")
    input_ids      = inputs.input_ids              ## .to("cuda")
    attention_mask = inputs.attention_mask    ## .to("cuda")

    outputs    = bert2bert.generate(input_ids, attention_mask=attention_mask)

    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred_summary"] = output_str

    return batch




In [76]:

batch_size = 16  # change to 64 for full evaluation

results = test_data.map(
               generate_summary, 
               batched=True, 
               batch_size=batch_size, 
               remove_columns=["article"]
)




Map:   0%|          | 0/230 [00:00<?, ? examples/s]

/Users/user/opt/anaconda3/envs/py37_HF2_Transformers/lib/python3.7/site-packages/transformers/generation/utils.py:1292: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [78]:

rouge.compute(
         predictions=results["pred_summary"], 
         references=results["highlights"], 
         rouge_types=["rouge2"]
)["rouge2"].mid


Score(precision=0.09515790074480426, recall=0.1531213063990967, fmeasure=0.11462285805329123)


## The model should achieve a ROUGE-2 score of 18.22 
